In [1]:
import pandas as pd
import scrapy
import time
from scrapy.crawler import CrawlerProcess

In [2]:
class eBaySpider(scrapy.Spider):
    name = "eBaySpider"
    start_urls = ['https://www.ebay-kleinanzeigen.de/s-autos/anzeige:angebote/seite:2/c216']
    
    def parse(self, response):
        urls = []
        for i in range(1, 3):
            urls.append(response.xpath('//*[@id="srchrslt-adtable-topads"]/li[{}]/article/div[2]/h2//a/@href'.format(str(i))).extract())
        for i in range(1, 28):
            urls.append(response.xpath('//*[@id="srchrslt-adtable"]/li[{}]/article/div[2]/h2//a/@href'.format(str(i))).extract())
        
        flat_urls = [item for sublist in urls for item in sublist]
        
        for url in flat_urls:
            url = response.urljoin(url)
            yield scrapy.Request(url=url, callback=self.parse_listing)
     
        next_page_url = response.xpath('//*[@id="srchrslt-pagination"]/div/div[3]/a/@href').extract_first()
        if next_page_url:
            next_page_url = response.urljoin(next_page_url)
            yield scrapy.Request(url=next_page_url, callback=self.parse)
    
    def parse_listing(self, response):
        gte = {"Besch\u00e4digtes Fahrzeug:" : "notRepairedDamage",
                "Marke:" : "brand",
                "Modell:" : "model",
                "Kilometerstand:" : "kilometer",
                'Erstzulassungsjahr:' : "yearOfRegistration",
                "Fahrzeugtyp:" : 'vehicleType',
                "Kraftstoffart:" : 'fuelType',
                "Leistung (PS):" : "powerPS",
                "Getriebe:" : "gearbox",
                "T\u00dcV Jahr:" : "someYearThing",
                "T\u00dcV Monat:": "monthThing",
                "Umweltplakette:" : "environmentalThing",
                "Schadstoffklasse:" : "emissionThing",
                "Au\u00dfenfarbe:": "color",
                "Material Innenausstattung:" : "interiorMaterial",
                "Anzahl Türen:" : "numberOfDoors",
                "Ausstattung:" : 'domesticEquipment',
                "Ort:" : "place",
                "Erstellungsdatum:" : 'creationDate',
                "Anzeigennummer:" : "referenceNumber",
                "Erstzulassungsmonat:": "monthOfRegistration",
              }
        
        for listing in response.xpath('//*[@id="viewad-main"]'):
            vari = {'name' : ''.join(listing.xpath("//*[@id='viewad-title']/text()").extract()).strip().lower(),
                'price' : ''.join(listing.xpath("//*[@id='viewad-price']/text()").extract()).strip().lower()
                  }
        for listing in response.xpath('//*[@id="viewad-details"]/section'):
            able = {gte[''.join(listing.xpath("//dl/dt[{}]/text()".format(i)).extract()).strip()] : ''.join(listing.xpath("//dl/dd[{}]/span/text()".format(i)).extract()).strip().lower()+''.join(listing.xpath("//dl/dd[{}]/span/a/text()".format(i)).extract()).strip().lower() for i in list(range(1,22))}
                  
        yield {**vari, **able}

In [3]:
start = time.time()
# Instantiate our crawler.
process = CrawlerProcess({
    'FEED_FORMAT': 'json',
    'FEED_URI': 'used_cars.json',
    'ROBOTSTXT_OBEY': True,
    'LOG_ENABLED': False,
    'AUTOTHROTTLE_ENABLED': True,
    'HTTPCACHE_ENABLED': True
})

# Start the crawler with our spider.
process.crawl(eBaySpider)
process.start()
print(time.time() - start)

56.82296109199524


## Importing the newly scraped dataset and making it match the old one.

In [4]:
used_cars = pd.read_json('used_cars.json')

In [5]:
autos = used_cars.copy()

In [6]:
autos['age'] = 2019 - autos.yearOfRegistration
autos = autos.drop(columns=['yearOfRegistration', 'color', 'creationDate', 'domesticEquipment', 'emissionThing', 'environmentalThing', 'interiorMaterial', 'monthOfRegistration', 'monthThing', 'referenceNumber', 'someYearThing', 'numberOfDoors', 'place'])

In [7]:
# Taking care of cleaning the price column
def clean_price(price):
    new_price = price.replace('preis: ', '')
    new_price = new_price.replace(' €', '')
    new_price = new_price.replace('.', '')
    new_price = new_price.replace(' vb', '')
    if new_price == '':
        return None
    else:
        new_price = int(new_price)
        return new_price

autos.price = autos.price.apply(lambda x: clean_price(x))
autos = autos.dropna(subset=['price'])

In [8]:
# Cleaning the kilometer column
autos.kilometer = autos.kilometer * 1000

def round_km(km):
    old_km_cats = [5000, 10000, 20000, 30000, 40000, 50000, 60000, 70000, 80000, 90000, 100000, 125000, 150000]
    return min(old_km_cats, key=lambda x:abs(x-km))

autos.kilometer = autos.kilometer.apply(lambda x: round_km(x))

In [9]:
# Removing accents from vehicleType column
autos.vehicleType = autos.vehicleType.apply(lambda x: x.replace('é', 'e'))

# Matching the values in vehicleType to the old format
def clean_type(vehicleType):
    if vehicleType == 'suv/geländewagen':
        return 'suv'
    elif vehicleType == 'van/bus':
        return 'bus'
    else: return vehicleType

autos.vehicleType = autos.vehicleType.apply(lambda x: clean_type(x))

In [10]:
# Matching the values in the fuelType column to the old format
def clean_fuel(fuel):
    if fuel == 'autogas (lpg)':
        return 'lpg'
    elif fuel == 'erdgas (cng)':
        return 'cng'
    else: return fuel
    
autos.fuelType = autos.fuelType.apply(lambda x: clean_fuel(x))

In [11]:
# Dropping all the values in brand/model that do not appear frequently enough in the training set

training_models = ['golf', 'a5', 'grand', 'fabia', '3er', '2_reihe', 'c_max',
       '3_reihe', 'passat', 'polo', 'twingo', 'a_klasse', 'scirocco',
       '5er', 'meriva', 'arosa', 'andere', 'civic', 'transporter',
       'punto', 'e_klasse', 'clio', 'kangoo', 'one', 'fortwo', '1er',
       'b_klasse', 'astra', 'a8', 'fiesta', 'c_klasse', 'micra', 'vito',
       'sprinter', '156', 'escort', 'scenic', 'a4', 'ka', 'a1',
       'insignia', 'focus', 'tt', 'corsa', 'a6', 'omega', 'slk', '7er',
       '80', '147', 'z_reihe', 'caddy', 'v40', 'ibiza', 'eos', 'touran',
       'a3', 'almera', 'megane', 'lupo', 'zafira', 'mondeo', 'colt',
       'vectra', 'berlingo', 'm_klasse', 'tiguan', 'i_reihe', 'espace',
       'sharan', '6_reihe', 'panda', 'seicento', 'octavia', 'fox',
       'beetle', 'touareg', 'x_reihe', 'galaxy', 'c3', 's_klasse',
       '1_reihe', 'avensis', 'sl', 'cooper', 'leon', '4_reihe', '500',
       'laguna', 'clk', 'primera', 'transit', 'qashqai', 'corolla',
       'swift', 'stilo', '911', 'mx_reihe', 'yaris', 'bora', 'tigra',
       'v70']

autos = autos[[i in training_models for i in autos.model]]

In [12]:
# Exporting it for use in the models notebook
autos.to_csv("new_used_cars.csv")